## Markdown to check number of features based on PCA

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib
from lime.lime_tabular import LimeTabularExplainer
from tqdm import tqdm

CLIP_FEATURES_DIR = "clip_features"
TRAIN = CLIP_FEATURES_DIR + "/train_features.pt"
VAL = CLIP_FEATURES_DIR + "/val_features.pt"
TOP_K_FEATURES = 100  # number of top features to keep based on SHAP importance
NUM_SAMPLES = 100

/home/tiago/thesis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_features(file_path):
    data = torch.load(file_path)
    return data["image_features"], data["text_features"], data["filenames"], data["labels"]


# Load train and validation features
train_img_features, train_txt_features, _, train_labels = load_features(TRAIN)
val_img_features, val_txt_features, _, val_labels = load_features(VAL)

In [3]:
# Combine image and text features for training
X_train = torch.cat((train_img_features, train_txt_features), dim=1)
X_val = torch.cat((val_img_features, val_txt_features), dim=1)

# Flatten features into a 2D matrix (samples x features)
X_train = X_train.view(X_train.size(0), -1).numpy()
X_val = X_val.view(X_val.size(0), -1).numpy()

# Convert labels to NumPy arrays
y_train = train_labels.numpy()
y_val = val_labels.numpy()     

# Standardize the features (important for PCA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)



## Train SVM

In [4]:
# Train a simple SVM model to get SHAP values
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train_scaled, y_train)


SVC(kernel='linear', probability=True)

In [ ]:
# Use LIME to compute feature importances over a subset of samples
explainer = LimeTabularExplainer(
    training_data=X_train_scaled,
    feature_names=[f"f{i}" for i in range(X_train_scaled.shape[1])],
    class_names=[str(c) for c in np.unique(y_train)],
    mode="classification"
)

# Limit the number of samples to speed up LIME processing
NUM_SAMPLES = 100
sample_indices = np.random.choice(X_train_scaled.shape[0], NUM_SAMPLES, replace=False)

feature_importance = np.zeros(X_train_scaled.shape[1])

for i in tqdm(sample_indices, desc="Computing LIME Explanations"):
    exp = explainer.explain_instance(X_train_scaled[i], svm_model.predict_proba, num_features=X_train_scaled.shape[1])
    for feat_idx, weight in exp.as_map()[1]:  # class 1 explanation
        feature_importance[feat_idx] += abs(weight)

# Normalize importances
feature_importance /= NUM_SAMPLES

IndexError: index 2019 is out of bounds for axis 0 with size 1024

<Figure size 1200x600 with 0 Axes>

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(range(len(feature_importance)), feature_importance)
plt.title("Feature Importance (Mean Absolute LIME Weight)")
plt.xlabel("Feature Index")
plt.ylabel("Average |Weight| from LIME")
plt.grid(True)
plt.show()

# %%
# Select top-k most important features
top_k_indices = np.argsort(feature_importance)[::-1][:TOP_K_FEATURES]
X_train_lime = X_train_scaled[:, top_k_indices]
X_val_lime = X_val_scaled[:, top_k_indices]

# Save indices for later use
#np.save("top_k_lime_indices.npy", top_k_indices)

# Save the scaler
#joblib.dump(scaler, "scaler_model.joblib")

# %%
# Train SVM on selected LIME features
svm_final = SVC(kernel='linear')
svm_final.fit(X_train_lime, y_train)

y_pred = svm_final.predict(X_val_lime)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy using Top {TOP_K_FEATURES} LIME-selected features: {accuracy:.4f}")

# Save model
#joblib.dump(svm_final, "svm_lime_model.joblib")